# A Short Walk Through Grid City

We're all at least intuitivey familiar with taxi-cab distance, aka Manhattan or $L^1$ distance. How many blocks will it take for us to walk from our start $s$ to our destination $t$?

(TAXI CAB IMG)

One clear consequence of the above metric is that it doesn't really matter how you traverse a grid; as long as you move towards your destination, going east then north takes as many blocks as moving northeast in a staircase-like fashion.

But, if you've ever lived in a city for an extended time, even one with a grid-like layout like the above, you might know it's not so simple. One missing piece from the simplified taxicab model above that lets us shave a few minutes off our morning commute to the bus stop is the saving time while crossing streets. In particular, not pictured in the schematic above are north/south (N/S) sidewalks for horizontal streets and east/west (E/W) sidewalks for vertical streets. You can only cross streets to get to the opposite sidewalk at intersections, and you might have to wait for the light to change.

Factoring this in, let's revisit the red blue paths from before, assuming $s$ is on the NE corner of its intersecion and $t$ is on the SE one.

(MULTIPLE CROSSINGS IMG)

Notice that going right first opens up a possibility to cross the street at $X$, but not one we're bound by. If there's a green light, we might cross, but could choose not to. On the other hand, if we want to follow the blue path then at point $Y$ we're obligated to cross north, so there's a risk we might be forced to wait at a light. Intuitively, it seems like we should be able to eke out some speed from the optionality of the red path.

I think in practice most pedestrians adopt the "opportunistic" approach: if there's a green light in one of the directions they're going, they'll cross the street. Is this optimal? How can we tell. Besides this overarching question, a few more pop up.

 1. Can we convince ourselves of this asymmetry? We need A More Refined Model.
 2. Can we quantify the opportunity? We should perform A Quantiative Analysis.
 3. Is there a deeper geometry which drives this behavior under the covers? Let's inspect the situation through A Manifold Approach.
 
## A More Refined Model

First, let's try modelling sidewalks in a graph of our example. Let's denote walking a block with a solid arrow and crossing an intersection with a dashed one.

(GRAPH CROSSINGS IMG)

Now the asymmetry is perhaps a bit more clear, but in addition we can notice some dissonance between our language referring to red and blue paths in the previous diagram: what the red path represented earlier was really a distribution (a flow, in some sense) over many paths we might take which start with a movement west.

However, we still have some work to do. It's natural to model the cost of walking a block to the time it takes us to do so. But what about the intersections? Naively, we might we tempted to treat each one as having an independent stochastic cost corresponding to some random variable for the wait time for a crosswalk. This would yield an easy shortest paths problem over a stochastic graph. We've solved such problems [on this blog before](https://vladfeinberg.com/2020/08/11/roller-coaster-tycoon-problem.html), they're just Markov Chains and have dynamic programming solutions. Unfortunately, this would be too crude of a simplification, costing us optionality: usually, if one direction has a red light, the other direction is green (or about to turn green). It's important to model the anti-correlation between N/S and E/W street crossing.

As a result, we'll need to do some more work in figuring out a way to represent this model. We'll have to do it in several steps:

 - First, we show that we can model each intersection in isolation. This independence is important because it lets us focus on just one intersection and we can repeat its modelling for other intesections without worrying about non-local effects.
 - Next, we will concern ourselves with only policies (decisions made by a pedestrian to cross or not cross) that make progress towards the objective. This will simplify what we have to model.
 - Finally, we make an explicit stoplight model and build its conditional probabilities into a Markov Chain which is the key to ending up with a tractable street traversal model.
 

### Treating Intersections Independently


Traffic engineers often coordinate stoplights between adjacent intersections so cars don't have to stop and go every single block. Doesn't this imply there's a nuanced dependence between when we can cross at one street corner vs another one block down?

At least for this post, I'd argue no. The main assumption we'll hinge on is that your walking speed is variable and  random, and that the standard deviation of time it takes you to walk a block is at least as large as the time it takes for a full stoplight cycle. That way, by the time you reach the next stoplight, it may as well be in a random place in its cycle of red and green lights. If a stoplight cycle is synchronized with a car traversing a block, and a car travels much faster than a person, then indeed we'd expect the person's time to walk the block to be much longer than a full stoplight cycle!

### Restriction to Monotone Policies

Without loss of generality, after shifting and rotating our viewpoint, we can assume our source $s$ starts at $(0, 0)$ and the destination $t$ is northeast of $s$.

Then to simplify modelling we'll only ever consider policies which, when greeted with an intersection, decide only between moving N, E, or NE. This wouldn't be realistic if we had to deal with obstacles which block intersections, forcing us to move around them inefficiently.

(CENTRAL PARK IMG)

But we're in Grid City, so that's not the case. Most importantly, we'll appeal to a notion of symmetry. If we ever try to cross the street S or W, it'll take just as long, on average, as the mirror N or E crossing. But on top of that, eventually we'll need to cross back north or east (maybe at a different intersection) to eventually catch up to the lattitude or longitude we were ate before. So there's strictly more time wasted in crossing streets in a direction away from our destination!

### An Explicit Stoplight Model

With all these preliminaries out of the way, we can focus on modelling the stochastic transition of the intersection towards the N or E. Since there's two ways of moving diagonally to the NE corner, we'll always only consider the fastest way of the two, whatever it may be (i.e., take the first green light across the first street).

(NORTH EAST FOUR AND THREE ARROW IMG)

But again, how do we model these arrows? The time you spend crossing to N or E or NE from the SW corner is correlated, and this breaks the typical [Markov Decision Process](https://en.wikipedia.org/wiki/Markov_decision_process) model, where cost must be a function of your state (here, we've implicitly associated with location) and action (which direction to go in). As-is, cost (time to wait to cross the street in a certain way) is a random variable anti-correlated between the different directions outlined in the diagram above, so it's not a deterministic function of state and action.

Instead, let's focus on the mechanics of the stoplight itself. Assume that it spends a unit of time total looping over four different states:

  - $A$. Can't cross either way, for instance because of a protected left turn for cars going N-then-W and S-then-E.
  - $B$. Can cross S to N.
  - $C$. Wait for the other pair of protected left turns.
  - $D$. Can cross W to E.
  
By our assumption $A+B+C+D=1$ for positive values $A, B, C, D$, which then also correspond as the probabilities that the stoplight is in the corresponding state upon arrival. Indeed, we can draw the corresponding markov chain for all the different situations, encoding the conditional structure of the above stoplight transitions, now respecting MDP structure. Below, edges are labelled by a pair `probability, cost`, where `probability` might be up to the policy (which tells us how to route, if we have a choice) that we are trying to find, e.g., $\pi_{\texttt{SW}, A}(\texttt{N only})$ denotes the chosen probability of choosing to cross N only (and not go diagonally to NE), assuming we arrived at the stoplight at state A. Furthermore, note that here we assume that the state of the stoplight cycle is observable. Cost is taken to be the expected cost, if there's any stochasticity (which must still meet MDP assumptions).

(ARRIVE AT INTERSECTION SW IMG)

Notice we have to model the decisions the agent makes seperately for all four cases. If we arrive at state $A$, we have to wait until we're in state $B$, on average this takes $A/2$ time. But if we then decide to move to the NE corner, we must wait for the entirety of the state $B$ to finish, taking $B$ time. But if we happened to arrive to the stoplight while $B$ was the state, on average we only need to wait $B/2$ time for state $C$ to start. We're also assuming we can cross intersections instantly.

If we arrive at the NW corner, our decisions are a bit more limited. Note that the state "arrive at intersection corner NW" and the state "NW" in the diagram above are distinct---otherwise, our pedestrian would effectively be seeing a new, randomly selected stoplight state after crossing from SW to NW corner, which loses the conditional structure we were hoping to build.

(ARRIVE AT INTERSECTION NW IMG)

Then the situation for arriving at the SE corner of the intersection is analogous to the above. Finally, because we're dealing with monotone policies, we never arrive at the NE corner of an intersection.

Parameterizations for $A, B, C, D$ allow us to quite flexibly model our stoplights. We could even make some of these values zero, or make the values different for different intersections.

All of this investment is finally paying off! Now we can do some computation :)

## A Quantitative Analysis

We derive a method which, given a policy, returns the expected runtime from $s$ to $t$. I'll gloss over the approach here since I've [previously discussed](https://vladfeinberg.com/2020/08/11/roller-coaster-tycoon-problem.html) how to do this using absorbing time. The key abstractions are:

  - parameterizing $m\times n$ rectangular chunks of grid city in terms of arrays $A, B, C, D$ corresponding to intersection timing parameters, where the $(0,0)$ index is the start $s$ and the $(m-1, n-1)$ index corresponds to the destination $t$. These are represented together with an $m\times n\times 4$ array.
  - In addition, the costs of walking the blocks is parameterized by east and north block costs, which are `(m-1, n)` and `(m, n-1)`-sized arrays corresponding to the cost of traversing the block immediately east and north of an intersection, respectively. These are aligned as sub-arrays of an $m \times n\times 2$ array
  - At every intersection there are 4 free parameters for the policy when arriving at each of the corners (SW, NW, SE) determining whether to only cross one block (in the SW case) or zero (in the NW/SE cases) instead of two or one, respectively, given we arrive at a given intersection stoplight state.
  - After crossing every intersection, the block policy tells us whether to travel north or east, if we are at the NE corner of the intersection (all other corners have only one direction to go in).
  
To simplify the code, we'll only ever consider starting at the SW corner and ending in the NE.

In [90]:
# E cost for blue vs red using absorbing (use "opportunistic" strategy for red)
# TODO -- separate github repo, pip install it. call helper functions here.
import numpy as np

def expected_absorption_time(stoplights, blocks, intersection_policy, block_policy):
    m, n, s = stoplights.shape
    assert s == 4, stoplights.shape # A, B, C, D
    assert blocks.shape == (m, n, 2) # east, north 
    assert intersection_policy.shape == (m, n, 3, 4) # SW, NW, SE x A, B, C, D
    assert block_policy.shape == (m, n) # probability of going east
    assert np.all(blocks[-1, :, 0] == 0) and np.all(blocks[:, -1, 1] == 0)
    
    t = {}  # transition dictionary, (from, to) -> (probability, cost)
    
    corners = ['SW', 'NW', 'SE']
    stoplight_states = ('A', 'B', 'C', 'D')
    
    # model intersections
    for i in range(m):
        for j in range(n):
            # For NW and SE corners, if the policy decides to move less, then we don't
            # cross any streets, which is easier to model.
            #
            # If we do decide to cross the street to go to NE, then on average
            # we wait for half the duration of the current stoplight state and
            # then the full duration of time for the following states until
            # it's the state which lets us cross. E.g., for NW, we need to wait until
            # state "D" to cross W to E, which the costs below encode the average
            # wait time for, conditional on the current stoplight state.
            movement_costs = {
                'NW': [[0.5, 1, 1, 0], [0, 0.5, 1, 0], [0, 0, 0.5, 0], [0] * 4],
                'SE': [[0] * 4, [0, 0.5, 1, 1], [0, 0, 0.5, 1], [0, 0, 0, 0.5]]
            }
            for corner, pi in zip(corners[1:], intersection_policy[i, j][1:]):
                start = (f'arrive {corner}', i, j)
                costs = movement_costs[corner]
                for stoplight, prob, pi_move_less, cost_factors in zip(
                    stoplight_states, stoplights[i, j], pi, costs):
                    dst = (f'{corner} {stoplight}', i, j)
                    t[(start, dst)] = (prob / stoplights[i, j].sum(), 0)
                    # Not crossing the street is free! 
                    t[(dst, (f'finish {corner}', i, j))] = (pi_move_less, 0)
                    t[(dst, ('finish NE', i, j))] = (1 - pi_move_less, stoplights[i, j].dot(cost_factors))
                    if (i == 0 and j == n - 1 and corner == 'NW') or (
                        i == m - 1 and j == 0 and corner == 'SE'):
                        assert pi_move_less == 0, "policy can't let you get stuck in corner"
            
            start = ('arrive SW', i, j)
            # Conditional on current stoplight state, what's the cost of crossing
            # the direction that's first available to us?
            one_block_costs = [[0.5, 0, 0, 0], [0] * 4, [0, 0, 0.5, 0], [0] * 4]
            # What's the average cost of crossing diagonally to NE?
            two_block_costs = [[0.5, 1, 1, 0], [0, 0.5, 1, 0],
                              [1, 0, 0.5, 1], [1, 0, 0, 0.5]]
            dst_1block = ['NW', 'NW', 'SE', 'SE']
            for stoplight, prob, pi_move_less, costs1b, costs2b, dst_move_less in zip(
                stoplight_states, stoplights[i, j], intersection_policy[i, j, 0],
                one_block_costs, two_block_costs, dst_1block):
                dst = (f'SW {stoplight}', i, j)
                t[(start, dst)] = (prob /  stoplights[i, j].sum(), 0)                    
                costs1b = stoplights[i, j].dot(costs1b)
                costs2b = stoplights[i, j].dot(costs2b)
                t[(dst, (f'finish {dst_move_less}', i, j))] = (pi_move_less, costs1b)
                t[(dst, (f'finish NE', i, j))] = (1 - pi_move_less, costs2b)
                if (i == 0 and j == n - 1 and dst_move_less == 'NW') or (
                    i == m - 1 and j == 0 and dst_move_less == 'SE'):
                    assert pi_move_less == 0, "policy can't let you get stuck in corner"
    
    # fixup the last intersection
    final = ('finish NE', m-1, n-1)
    t[(final, final)] = (1, 0)
    
    # add east blocks
    for i in range(m - 1):
        for j in range(n):
            src = ('finish NE', i, j)
            dst = ('arrive NW', i + 1, j)
            t[(src, dst)] = (block_policy[i, j], blocks[i, j, 0])
            if j == n - 1:
                assert block_policy[i, j] == 1
            src = ('finish SE', i, j)
            dst = ('arrive SW', i + 1, j)
            t[(src, dst)] = (1, blocks[i, j, 0])
            
    # add north blocks
    for i in range(m):
        for j in range(n - 1):
            src = ('finish NE', i, j)
            dst = ('arrive SE', i, j + 1)
            t[(src, dst)] = (1 - block_policy[i, j], blocks[i, j, 1])
            if i == m - 1:
                assert block_policy[i, j] == 0
            src = ('finish NW', i, j)
            dst = ('arrive SW', i, j + 1)
            t[(src, dst)] = (1, blocks[i, j, 1])
            
    states = {k for src, dst in t for k in (src, dst)}
    states = {k: i for i, k in enumerate(states)}
    N = len(states)
    P = np.zeros((N, N))
    C = np.zeros((N, N))
    for (src, dst), (prob, cost) in t.items():
        P[states[src], states[dst]] = prob
        C[states[src], states[dst]] = cost
    assert np.all(C >= 0)
    
    while True:
        invstates = {i: k for k, i in states.items()}
        noexit = np.flatnonzero(0 == P.sum(axis=1))
        child = {}
        leads_to_no_exit = set()
        parents = set(noexit)
        while True:
            new_parents = set()
            for i in parents:
                to_prune = {j for j in np.flatnonzero(P[:, i])}
                new_parents |= to_prune
                for p in to_prune:
                    if p not in child:
                        child[p] = i
            new_parents = new_parents - leads_to_no_exit
            parents = new_parents
            leads_to_no_exit |= new_parents
            if not new_parents:
                break
            start = states[('arrive SW', 0, 0)]
            if start in leads_to_no_exit:
                path = ''
                i = start
                while P[i, :].sum():
                    path += ' ' + invstates[i]
                    i = child[i]
                raise ValueError('policy leads to non-absorbing state:' + path)

        print('pruning', len(leads_to_no_exit), 'dead states')

        keep = np.ones(len(P),dtype=bool)
        for i in leads_to_no_exit:
            keep[i] = 0

        pruned = [invstates[i] for i in leads_to_no_exit]
        print(pruned)

        P = P[keep][:, keep]
        C = C[keep][:, keep]

        keep_ix = np.add.accumulate(keep) - 1
        states = {k: keep_ix[i] for k, i in states.items() if keep[i]}
        if len(leads_to_no_exit) == 0:
            break
            
    #noexit = np.flatnonzero(0 == P.sum(axis=1))
    #assert np.all(P.sum(axis=0)[noexit] == 0)
    
    for i, row in enumerate(P):
        assert row.sum() == 1, (i, row.sum(), [k for k, ii in states.items() if ii == i][0])
    # Policy evaluation formula for v, the value function under the
    # current policy
    # v = (P * C) 1 + P v
    # (I - P) v = (P * C) 1
    v, res, rank, _ = np.linalg.lstsq(np.eye(N) - P, (P * C).dot(np.ones(N)), rcond=None)
    assert np.all(v >= 0)
    assert not len(res), rank
    print(np.linalg.norm((np.eye(N) - P).dot(v) - (P * C).dot(np.ones(N))))
    return v[states[('arrive SW', 0, 0)]], v, states, pruned

In [91]:
# 1 block across, 2 blocks up case

stoplights = np.ones((2, 3, 4))
blocks = np.ones((2, 3, 2)) * 10
blocks[-1, :, 0] = 0
blocks[:, -1, 1] = 0

# blue policy
intersection_policy = np.ones((2, 3, 3, 4))
block_policy = np.zeros((2, 3))
block_policy[:, -1] = 1  # must go east at top
SW, NW, SE = 0, 1, 2
A, B, C, D = 0, 1, 2, 3
# At (0, 0), blue policy moves north. So it is OK moving less when NW and SW, but not SE.
intersection_policy[0, 0, SE] = 0
intersection_policy[0, 0, NW, D] = 0  # Still, if we're in NW corner, and W->E is open, cross it, it's free.
intersection_policy[0, 0, SW, [C, D]] = 0  # If in SW corner, and we cross W->E, we must cross S->N too.

# At (0, 1), blue policy moves north.
intersection_policy[0, 1] = intersection_policy[0, 0]

# At (0, 2), blue policy goes east, but can cross streets optimistically/greedily.
intersection_policy[0, 2, SE, B] = 0
intersection_policy[0, 2, NW, D] = 0

# Same for (1, 2)
intersection_policy[1, 2] = intersection_policy[0, 1]

# Other parts of policy don't matter; they should never get visited.

# Don't get stuck in corners. Always cross streets in these situations.
intersection_policy[0, -1, NW] = 0
intersection_policy[0, -1, SW, [A, B]] = 0
intersection_policy[-1, 0, SE] = 0
intersection_policy[-1, 0, SW, [C, D]] = 0

# At the top left corner you have to get to the NE.
intersection_policy[-1, -1] = 0

t, v, st, p = expected_absorption_time(stoplights, blocks, intersection_policy, block_policy)

print(p)
# should be zero
sum(v[st[k]] for k in st if k[1:] not in ((0, 0), (0, 1), (0, 2), (1, 2))), t

pruning 27 dead states
[('SW C', 1, 0), ('SE A', 1, 1), ('NW D', 1, 0), ('SW D', 1, 0), ('SE B', 1, 1), ('SW B', 1, 0), ('SE C', 1, 0), ('arrive SW', 1, 1), ('NW B', 1, 0), ('NW A', 1, 0), ('SE B', 1, 0), ('arrive SE', 1, 0), ('SW C', 1, 1), ('finish SE', 0, 0), ('arrive SE', 1, 1), ('NW C', 1, 0), ('SE D', 1, 0), ('SW D', 1, 1), ('SE A', 1, 0), ('arrive NW', 1, 0), ('finish SE', 0, 1), ('SE C', 1, 1), ('SW A', 1, 0), ('finish NE', 1, 0), ('arrive SW', 1, 0), ('finish NW', 1, 0), ('SE D', 1, 1)]
pruning 0 dead states
[]


AssertionError: (4, 0.0, ('finish NW', 0, 2))

In [15]:
# optimal policy
import numpy as np
help(np.linalg.lstsq)

Help on function lstsq in module numpy.linalg:

lstsq(a, b, rcond='warn')
    Return the least-squares solution to a linear matrix equation.
    
    Computes the vector `x` that approximatively solves the equation
    ``a @ x = b``. The equation may be under-, well-, or over-determined
    (i.e., the number of linearly independent rows of `a` can be less than,
    equal to, or greater than its number of linearly independent columns).
    If `a` is square and of full rank, then `x` (but for round-off error)
    is the "exact" solution of the equation. Else, `x` minimizes the
    Euclidean 2-norm :math:`||b - ax||`. If there are multiple minimizing 
    solutions, the one with the smallest 2-norm :math:`||x||` is returned.
    
    Parameters
    ----------
    a : (M, N) array_like
        "Coefficient" matrix.
    b : {(M,), (M, K)} array_like
        Ordinate or "dependent variable" values. If `b` is two-dimensional,
        the least-squares solution is calculated for each of the `K